In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import time

In [2]:
import os
from dotenv import load_dotenv 
load_dotenv()

True

In [ ]:
load_dotenv()
tmdb_key = os.getenv("tmdb_key")

In [ ]:
load_dotenv()
tmdb_access_token = os.getenv("tmdb_access_token")

In [18]:
bechdel_new_df = pd.read_csv('../data/Bechdel_updated.csv',index_col=0)
bechdel_new_df.drop(columns=['Unnamed: 0'], inplace=True)
bechdel_new_df.sample(4)

,title,year,rating,dubious,imdbid,id,submitterid,date,visible
195,"Turning, The",2020.0,3.0,0.0,7510346.0,9003.0,17695.0,2020-02-06 16:50:54,1.0
438,Night Teeth,2021.0,2.0,0.0,10763820.0,10114.0,19672.0,2021-10-31 20:47:12,1.0
67,Can You Keep A Secret?,2019.0,3.0,0.0,8707922.0,9027.0,17758.0,2020-03-03 04:50:27,1.0
430,"Many Saints of Newark, The",2021.0,2.0,0.0,8110232.0,10074.0,19583.0,2021-10-03 22:25:04,1.0


In [ ]:
# let's check with values i should get from TMDB
movies = pd.read_csv("../data/movies.csv",index_col=0)
movies.info()

In [ ]:
# tmbdid, genres, genres, popularity, production_companies, production_countries, release_date, revenue, vote_average  
# vote_count, crew, budget, cast_gender, crew_gender, cast_female_representation, crew_female_representation 

In [ ]:
bechdel_new_df = bechdel_new_df.rename(columns={'rating': 'bt_score'})

In [ ]:
bechdel_id = list(bechdel_new_df["imdbid"])
movies_id = list(movies["imdbid"])
movies_id = [el for el in bechdel_id if el not in movies_id]
len(movies_id)

In [ ]:
#movies_id

In [ ]:
# change imdbit in the format ddddddd
def get_id(dep_id):
    try:
        dep_id = str(int(dep_id))
        l = len(dep_id)
        new_id = '0'*(7-l)+dep_id
        return new_id
    except:
        pass
    
def change_all_id(lista):
    for i in range(len(lista)):
        lista[i] = get_id(lista[i])
    return lista

In [ ]:
movies_id = change_all_id(movies_id)

In [ ]:
#movies_id

In [ ]:
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_access_token}"
}

In [ ]:
credit_url = f"https://api.themoviedb.org/3/movie/tt9059704/credits?language=en-US"
credit_response = requests.get(credit_url, headers=headers).json()
credit_response

In [ ]:
cast = credit_response['cast']
crew = credit_response['crew']

In [ ]:
details_url = f"https://api.themoviedb.org/3/movie/tt12198524?language=en-US"
response = requests.get(details_url, headers=headers).json()
response

In [ ]:
genres = [el["name"] for el in response["genres"]]
imdb_id = response["imdb_id"].replace('tt','')
budget = response["budget"]
popularity = response["popularity"]
production_companies = response['production_companies']
release_date = response['release_date']
revenue = response['revenue']
vote_average = response['vote_average']
vote_count = response['vote_count']

In [ ]:
def get_movie_info(imdbid_list):
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {tmdb_access_token}"
    }
    
    new_movie_list = []
    cont=0
    
    for imbdid in imdbid_list:
        movie_dict={}
        time.sleep(1)
        details_url = f"https://api.themoviedb.org/3/movie/tt{imbdid}?language=en-US"        
        credits_url = f"https://api.themoviedb.org/3/movie/tt{imbdid}/credits?language=en-US"
        
        
        details_response = requests.get(details_url, headers=headers).json()
        try: 
            movie_dict["imdbid"] = imbdid
            movie_dict["genres"] = [el["name"] for el in details_response["genres"]]           
            movie_dict["budget"] = details_response["budget"]
            movie_dict["popularity"] = details_response["popularity"]
            movie_dict["production_companies"] = details_response['production_companies']
            movie_dict["release_date"] = details_response['release_date']
            movie_dict["revenue"] = details_response['revenue']
            movie_dict["vote_average"] = details_response['vote_average']
            movie_dict["vote_count"] = details_response['vote_count']
        except:
            pass
    
        credit_response = requests.get(credits_url, headers=headers).json()
        
        try:
            movie_dict['cast'] = credit_response['cast']
            movie_dict['crew'] = credit_response['crew']
        except:
            pass
        
        new_movie_list.append(movie_dict)
        
        if cont%20==0:
            pd.DataFrame(new_movie_list).to_csv("../data/checkpoint_df.csv",index=False)
        cont+=1
        
    return new_movie_list

In [ ]:
movie_list = get_movie_info(movies_id)

In [7]:
df = pd.read_csv("../data/checkpoint_df.csv")
df

,genres,imdbid,budget,popularity,production_companies,release_date,revenue,vote_average,vote_count,cast,crew
0,"['Science Fiction', 'Action', 'Thriller']",1025100.0,140000000.0,40.054,"[{'id': 82819, 'logo_path': '/gXfFl9pRPaoaq14j...",2019-10-02,173469516.0,6.313,4778.0,"[{'adult': False, 'gender': 2, 'id': 2888, 'kn...","[{'adult': False, 'gender': 1, 'id': 2952, 'kn..."
1,"['Drama', 'Action', 'History']",1950186.0,97600000.0,64.900,"[{'id': 25, 'logo_path': '/qZCc1lty5FzX30aOCVR...",2019-11-13,225508210.0,8.012,7078.0,"[{'adult': False, 'gender': 2, 'id': 3894, 'kn...","[{'adult': False, 'gender': 1, 'id': 1720, 'kn..."
2,"['Drama', 'Comedy', 'Crime']",5503686.0,20000000.0,33.157,"[{'id': 13184, 'logo_path': '/pfUB1a62jSMIqp4X...",2019-09-12,157600000.0,6.487,2773.0,"[{'adult': False, 'gender': 1, 'id': 206444, '...","[{'adult': False, 'gender': 1, 'id': 3429, 'kn..."
3,"['Family', 'Fantasy', 'Adventure', 'Action']",4777008.0,185000000.0,46.907,"[{'id': 2, 'logo_path': '/wdrCwmRnLFJhEoH8GSfy...",2019-10-16,491730089.0,7.321,5749.0,"[{'adult': False, 'gender': 1, 'id': 11701, 'k...","[{'adult': False, 'gender': 2, 'id': 1722, 'kn..."
4,"['Drama', 'Crime', 'Comedy']",5865326.0,0.0,15.499,"[{'id': 10039, 'logo_path': '/b8w4LldQolMKiLZw...",2019-09-27,0.0,6.004,1120.0,"[{'adult': False, 'gender': 1, 'id': 5064, 'kn...","[{'adult': False, 'gender': 2, 'id': 1884, 'kn..."
...,...,...,...,...,...,...,...,...,...,...,...
636,['Comedy'],14309446.0,0.0,32.523,"[{'id': 160027, 'logo_path': None, 'name': 'Pa...",2022-08-26,0.0,5.927,770.0,"[{'adult': False, 'gender': 2, 'id': 55638, 'k...","[{'adult': False, 'gender': 1, 'id': 17150, 'k..."
637,"['Drama', 'Romance']",13139228.0,0.0,18.508,"[{'id': 20580, 'logo_path': '/oRR9EXVoKP9szDkV...",2022-10-20,0.0,7.764,592.0,"[{'adult': False, 'gender': 2, 'id': 1052109, ...","[{'adult': False, 'gender': 1, 'id': 471, 'kno..."
638,"['Adventure', 'Mystery', 'Crime']",14641788.0,0.0,37.480,"[{'id': 923, 'logo_path': '/5UQsZrfbfG2dYJbx8D...",2022-11-04,0.0,7.536,2206.0,"[{'adult': False, 'gender': 1, 'id': 1356210, ...","[{'adult': False, 'gender': 2, 'id': 3079, 'kn..."
639,"['Comedy', 'Romance']",15218000.0,10000000.0,14.970,"[{'id': 24021, 'logo_path': '/urq2p5ColCf657b5...",2022-06-09,0.0,7.162,195.0,"[{'adult': False, 'gender': 2, 'id': 2371545, ...","[{'adult': False, 'gender': 2, 'id': 19759, 'k..."


In [ ]:
#df = pd.DataFrame(movie_list)
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)

In [16]:
df

,imdbid,genres,budget,popularity,production_companies,release_date,revenue,vote_average,vote_count,cast,crew
0,1025100.0,"['Science Fiction', 'Action', 'Thriller']",140000000.0,40.054,"[{'id': 82819, 'logo_path': '/gXfFl9pRPaoaq14j...",2019-10-02,173469516.0,6.313,4778.0,"[{'adult': False, 'gender': 2, 'id': 2888, 'kn...","[{'adult': False, 'gender': 1, 'id': 2952, 'kn..."
1,1950186.0,"['Drama', 'Action', 'History']",97600000.0,64.900,"[{'id': 25, 'logo_path': '/qZCc1lty5FzX30aOCVR...",2019-11-13,225508210.0,8.012,7078.0,"[{'adult': False, 'gender': 2, 'id': 3894, 'kn...","[{'adult': False, 'gender': 1, 'id': 1720, 'kn..."
2,5503686.0,"['Drama', 'Comedy', 'Crime']",20000000.0,33.157,"[{'id': 13184, 'logo_path': '/pfUB1a62jSMIqp4X...",2019-09-12,157600000.0,6.487,2773.0,"[{'adult': False, 'gender': 1, 'id': 206444, '...","[{'adult': False, 'gender': 1, 'id': 3429, 'kn..."
3,4777008.0,"['Family', 'Fantasy', 'Adventure', 'Action']",185000000.0,46.907,"[{'id': 2, 'logo_path': '/wdrCwmRnLFJhEoH8GSfy...",2019-10-16,491730089.0,7.321,5749.0,"[{'adult': False, 'gender': 1, 'id': 11701, 'k...","[{'adult': False, 'gender': 2, 'id': 1722, 'kn..."
4,5865326.0,"['Drama', 'Crime', 'Comedy']",0.0,15.499,"[{'id': 10039, 'logo_path': '/b8w4LldQolMKiLZw...",2019-09-27,0.0,6.004,1120.0,"[{'adult': False, 'gender': 1, 'id': 5064, 'kn...","[{'adult': False, 'gender': 2, 'id': 1884, 'kn..."
...,...,...,...,...,...,...,...,...,...,...,...
609,14309446.0,['Comedy'],0.0,32.523,"[{'id': 160027, 'logo_path': None, 'name': 'Pa...",2022-08-26,0.0,5.927,770.0,"[{'adult': False, 'gender': 2, 'id': 55638, 'k...","[{'adult': False, 'gender': 1, 'id': 17150, 'k..."
610,13139228.0,"['Drama', 'Romance']",0.0,18.508,"[{'id': 20580, 'logo_path': '/oRR9EXVoKP9szDkV...",2022-10-20,0.0,7.764,592.0,"[{'adult': False, 'gender': 2, 'id': 1052109, ...","[{'adult': False, 'gender': 1, 'id': 471, 'kno..."
611,14641788.0,"['Adventure', 'Mystery', 'Crime']",0.0,37.480,"[{'id': 923, 'logo_path': '/5UQsZrfbfG2dYJbx8D...",2022-11-04,0.0,7.536,2206.0,"[{'adult': False, 'gender': 1, 'id': 1356210, ...","[{'adult': False, 'gender': 2, 'id': 3079, 'kn..."
612,15218000.0,"['Comedy', 'Romance']",10000000.0,14.970,"[{'id': 24021, 'logo_path': '/urq2p5ColCf657b5...",2022-06-09,0.0,7.162,195.0,"[{'adult': False, 'gender': 2, 'id': 2371545, ...","[{'adult': False, 'gender': 2, 'id': 19759, 'k..."


In [26]:
# junta los 3 dataframes y elimina columnas duplicadas
# bechdel_new_df = pd.read_csv("../data/Bechdel_updated.csv")

new_movies_df = pd.merge(bechdel_new_df, df, left_on='imdbid', right_on='imdbid', how='inner')

new_movies_df

,title,year,rating,dubious,imdbid,id,submitterid,date,visible,genres,budget,popularity,production_companies,release_date,revenue,vote_average,vote_count,cast,crew
0,Gemini Man,2019.0,1.0,0.0,1025100.0,8868.0,17383.0,2019-10-12 10:46:05,1.0,"['Science Fiction', 'Action', 'Thriller']",140000000.0,40.054,"[{'id': 82819, 'logo_path': '/gXfFl9pRPaoaq14j...",2019-10-02,173469516.0,6.313,4778.0,"[{'adult': False, 'gender': 2, 'id': 2888, 'kn...","[{'adult': False, 'gender': 1, 'id': 2952, 'kn..."
1,Ford v Ferrari,2019.0,0.0,0.0,1950186.0,8869.0,17384.0,2019-10-12 10:48:31,1.0,"['Drama', 'Action', 'History']",97600000.0,64.900,"[{'id': 25, 'logo_path': '/qZCc1lty5FzX30aOCVR...",2019-11-13,225508210.0,8.012,7078.0,"[{'adult': False, 'gender': 2, 'id': 3894, 'kn...","[{'adult': False, 'gender': 1, 'id': 1720, 'kn..."
2,Hustlers,2019.0,3.0,0.0,5503686.0,8870.0,17389.0,2019-10-14 13:49:26,1.0,"['Drama', 'Comedy', 'Crime']",20000000.0,33.157,"[{'id': 13184, 'logo_path': '/pfUB1a62jSMIqp4X...",2019-09-12,157600000.0,6.487,2773.0,"[{'adult': False, 'gender': 1, 'id': 206444, '...","[{'adult': False, 'gender': 1, 'id': 3429, 'kn..."
3,Maleficent: Mistress of Evil,2019.0,3.0,0.0,4777008.0,8876.0,17403.0,2019-10-26 22:39:21,1.0,"['Family', 'Fantasy', 'Adventure', 'Action']",185000000.0,46.907,"[{'id': 2, 'logo_path': '/wdrCwmRnLFJhEoH8GSfy...",2019-10-16,491730089.0,7.321,5749.0,"[{'adult': False, 'gender': 1, 'id': 11701, 'k...","[{'adult': False, 'gender': 2, 'id': 1722, 'kn..."
4,"Laundromat, The",2019.0,3.0,0.0,5865326.0,8877.0,17405.0,2019-10-27 07:11:37,1.0,"['Drama', 'Crime', 'Comedy']",0.0,15.499,"[{'id': 10039, 'logo_path': '/b8w4LldQolMKiLZw...",2019-09-27,0.0,6.004,1120.0,"[{'adult': False, 'gender': 1, 'id': 5064, 'kn...","[{'adult': False, 'gender': 2, 'id': 1884, 'kn..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,Me Time,2022.0,1.0,0.0,14309446.0,10596.0,20571.0,2022-11-04 19:59:24,1.0,['Comedy'],0.0,32.523,"[{'id': 160027, 'logo_path': None, 'name': 'Pa...",2022-08-26,0.0,5.927,770.0,"[{'adult': False, 'gender': 2, 'id': 55638, 'k...","[{'adult': False, 'gender': 1, 'id': 17150, 'k..."
610,My Policeman,2022.0,2.0,0.0,13139228.0,10598.0,20576.0,2022-11-06 07:42:25,1.0,"['Drama', 'Romance']",0.0,18.508,"[{'id': 20580, 'logo_path': '/oRR9EXVoKP9szDkV...",2022-10-20,0.0,7.764,592.0,"[{'adult': False, 'gender': 2, 'id': 1052109, ...","[{'adult': False, 'gender': 1, 'id': 471, 'kno..."
611,Enola Holmes 2,2022.0,3.0,0.0,14641788.0,10600.0,20579.0,2022-11-08 06:02:21,1.0,"['Adventure', 'Mystery', 'Crime']",0.0,37.480,"[{'id': 923, 'logo_path': '/5UQsZrfbfG2dYJbx8D...",2022-11-04,0.0,7.536,2206.0,"[{'adult': False, 'gender': 1, 'id': 1356210, ...","[{'adult': False, 'gender': 2, 'id': 3079, 'kn..."
612,Fire Island,2022.0,3.0,0.0,15218000.0,10601.0,20581.0,2022-11-08 13:23:26,1.0,"['Comedy', 'Romance']",10000000.0,14.970,"[{'id': 24021, 'logo_path': '/urq2p5ColCf657b5...",2022-06-09,0.0,7.162,195.0,"[{'adult': False, 'gender': 2, 'id': 2371545, ...","[{'adult': False, 'gender': 2, 'id': 19759, 'k..."


In [ ]:
# function to get cast/crew gender

In [27]:
for index, movie in new_movies_df.iterrows():
    companies = eval(movie['production_companies'])
    companies_list = []

    for company in companies:
        company_name = company['name']
        companies_list.append(company_name)

    new_movies_df.at[index, 'production_companies'] = companies_list

In [28]:
for index, movie in new_movies_df.iterrows():
    date = datetime.strptime(movie['release_date'], '%Y-%m-%d')
    reformatted_date = date.strftime('%d/%m/%Y')

    new_movies_df.at[index, 'release_date'] = reformatted_date

In [30]:
new_movies_df['cast_gender'] = pd.NA


for index, movie in new_movies_df.iterrows():
    cast = eval(movie['cast'])
    gender_list = []

    for cast_member in cast:
        gender = cast_member['gender']
        gender_list.append(gender)

    new_movies_df.at[index, 'cast_gender'] = gender_list

In [31]:
new_movies_df['crew_gender'] = pd.NA


for index, movie in new_movies_df.iterrows():
    crew = eval(movie['crew'])
    gender_list = []

    for crew_member in crew:
        gender = crew_member['gender']
        gender_list.append(gender)

    new_movies_df.at[index, 'crew_gender'] = gender_list

In [32]:
new_movies_df['cast_female_representation'] = pd.NA
new_movies_df['crew_female_representation'] = pd.NA


for index, movie in new_movies_df.iterrows():
    if len(movie['cast_gender']) != 0:
        new_movies_df.at[index, 'cast_female_representation'] = 100*(movie['cast_gender'].count(1)/len(movie['cast_gender']))

    if len(movie['crew_gender']) != 0:
        new_movies_df.at[index, 'crew_female_representation'] = 100*(movie['crew_gender'].count(1)/len(movie['crew_gender']))


new_movies_df = new_movies_df.dropna().copy()
new_movies_df['cast_female_representation'] = new_movies_df['cast_female_representation'].astype('float64')
new_movies_df['crew_female_representation'] = new_movies_df['crew_female_representation'].astype('float64')

In [33]:
new_movies_df

,title,year,rating,dubious,imdbid,id,submitterid,date,visible,genres,...,release_date,revenue,vote_average,vote_count,cast,crew,cast_gender,crew_gender,cast_female_representation,crew_female_representation
0,Gemini Man,2019.0,1.0,0.0,1025100.0,8868.0,17383.0,2019-10-12 10:46:05,1.0,"['Science Fiction', 'Action', 'Thriller']",...,02/10/2019,173469516.0,6.313,4778.0,"[{'adult': False, 'gender': 2, 'id': 2888, 'kn...","[{'adult': False, 'gender': 1, 'id': 2952, 'kn...","[2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, ...",27.450980,4.593640
1,Ford v Ferrari,2019.0,0.0,0.0,1950186.0,8869.0,17384.0,2019-10-12 10:48:31,1.0,"['Drama', 'Action', 'History']",...,13/11/2019,225508210.0,8.012,7078.0,"[{'adult': False, 'gender': 2, 'id': 3894, 'kn...","[{'adult': False, 'gender': 1, 'id': 1720, 'kn...","[2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, ...",6.250000,10.238908
2,Hustlers,2019.0,3.0,0.0,5503686.0,8870.0,17389.0,2019-10-14 13:49:26,1.0,"['Drama', 'Comedy', 'Crime']",...,12/09/2019,157600000.0,6.487,2773.0,"[{'adult': False, 'gender': 1, 'id': 206444, '...","[{'adult': False, 'gender': 1, 'id': 3429, 'kn...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, ...","[1, 0, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 0, 1, ...",50.000000,31.666667
3,Maleficent: Mistress of Evil,2019.0,3.0,0.0,4777008.0,8876.0,17403.0,2019-10-26 22:39:21,1.0,"['Family', 'Fantasy', 'Adventure', 'Action']",...,16/10/2019,491730089.0,7.321,5749.0,"[{'adult': False, 'gender': 1, 'id': 11701, 'k...","[{'adult': False, 'gender': 2, 'id': 1722, 'kn...","[1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, ...","[2, 2, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 1, 1, 1, ...",19.277108,10.989011
4,"Laundromat, The",2019.0,3.0,0.0,5865326.0,8877.0,17405.0,2019-10-27 07:11:37,1.0,"['Drama', 'Crime', 'Comedy']",...,27/09/2019,0.0,6.004,1120.0,"[{'adult': False, 'gender': 1, 'id': 5064, 'kn...","[{'adult': False, 'gender': 2, 'id': 1884, 'kn...","[1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, ...","[2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, ...",26.153846,17.094017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,Me Time,2022.0,1.0,0.0,14309446.0,10596.0,20571.0,2022-11-04 19:59:24,1.0,['Comedy'],...,26/08/2022,0.0,5.927,770.0,"[{'adult': False, 'gender': 2, 'id': 55638, 'k...","[{'adult': False, 'gender': 1, 'id': 17150, 'k...","[2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, ...","[1, 2, 2, 2, 1, 2, 2, 2, 1, 0, 2, 1, 1, 2, 0, ...",25.806452,29.411765
610,My Policeman,2022.0,2.0,0.0,13139228.0,10598.0,20576.0,2022-11-06 07:42:25,1.0,"['Drama', 'Romance']",...,20/10/2022,0.0,7.764,592.0,"[{'adult': False, 'gender': 2, 'id': 1052109, ...","[{'adult': False, 'gender': 1, 'id': 471, 'kno...","[2, 3, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 0, ...","[1, 2, 2, 0, 2, 1, 0, 2, 2, 0, 0, 2, 1, 2, 2, ...",17.391304,9.375000
611,Enola Holmes 2,2022.0,3.0,0.0,14641788.0,10600.0,20579.0,2022-11-08 06:02:21,1.0,"['Adventure', 'Mystery', 'Crime']",...,04/11/2022,0.0,7.536,2206.0,"[{'adult': False, 'gender': 1, 'id': 1356210, ...","[{'adult': False, 'gender': 2, 'id': 3079, 'kn...","[1, 2, 2, 2, 1, 2, 1, 1, 1, 0, 2, 1, 1, 0, 2, ...","[2, 1, 1, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, ...",14.285714,16.216216
612,Fire Island,2022.0,3.0,0.0,15218000.0,10601.0,20581.0,2022-11-08 13:23:26,1.0,"['Comedy', 'Romance']",...,09/06/2022,0.0,7.162,195.0,"[{'adult': False, 'gender': 2, 'id': 2371545, ...","[{'adult': False, 'gender': 2, 'id': 19759, 'k...","[2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, ...","[2, 1, 2, 2, 0, 0, 2, 2, 2, 0, 2, 1, 2, 0, 0, ...",10.714286,10.937500


In [34]:
new_movies_df.to_csv("../data/new_movies.csv")